In [ ]:
# 1. Install dependency

# 1) Upgrade pip
!pip install -q --upgrade pip

# 2) Synchronize huggingface-hub and fsspec versions
!pip install -q fsspec==2025.3.2 huggingface-hub>=0.28.1

# 3) Install torch/torchvision/torchaudio
!pip install -q torch torchvision torchaudio

# 4) Install the rest of the dependencies
!pip install -q \
  transformers datasets peft accelerate bitsandbytes sentencepiece einops gdown tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 185.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 161.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 155.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 163.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 174.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 164.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [nvidia-cusolver-cu12]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# 2. # Log in to Hugging Face Hub from the notebook
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
# 3. List contents of the Follow-upQA folder
!pip install -q pydrive2

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1) Authenticate in Colab
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

# 2) Shared folder ID
folder_id = "1sdgi9g5uuXARsLzveDwoeoi4xKf0eg4V"

# 3) List files
query = f"'{folder_id}' in parents and trashed=false"
file_list = gdrive.ListFile({'q': query}).GetList()

print("Files in Follow-upQA folder:")
for f in file_list:
    print(f"• {f['title']}    (ID: {f['id']})")


Files in Follow-upQA folder:
• Follow-upQA    (ID: 1g_EcRzT4sEJ1-LATG2F6IiozuIWkMbU4)
• passage_emb    (ID: 1lpoHBYmWJe7-jWB6KPwC0amM8CK8CLl7)
• KGs    (ID: 1bSwzX4KViZOev2HjPc1Tcy07G3P2Jqj4)
• HotpotQA    (ID: 1SZWdSWhomEtaMFsG5k5wOZ1aFb7DDZWA)
• 2WikiMQA    (ID: 1PLoZwIa6Xt7mg0nDpnS-frtcmMsykFFv)


In [ ]:
# 4. List contents of the Follow-upQA subfolder
folder_id = "1g_EcRzT4sEJ1-LATG2F6IiozuIWkMbU4"  # the Follow-upQA folder

query = f"'{folder_id}' in parents and trashed=false"
subfiles = gdrive.ListFile({'q': query}).GetList()

print("Files inside Follow-upQA folder:")
for f in subfiles:
    print(f"• {f['title']}    (ID: {f['id']})")


Files inside Follow-upQA folder:
• mistral_train_val_test    (ID: 1kvcYB018Z_DXdBtJ1NFBrNXrPwDamgOY)
• follow-upQA.json    (ID: 1qAVZOlyLXk4XCjONcyIhCkU0Buc98c2-)


In [ ]:
# 5. Download and preview the follow-upQA.json dataset from Google Drive
import gdown, os, json, pandas as pd, random, shutil

# Working directory
DATA_DIR = "/content/data"
os.makedirs(DATA_DIR, exist_ok=True)

# Download the file from Google Drive
file_id = "1qAVZOlyLXk4XCjONcyIhCkU0Buc98c2-"   # follow-upQA.json
gdown.download(id=file_id, output=f"{DATA_DIR}/follow-upQA.json", quiet=False)

with open(f"{DATA_DIR}/follow-upQA.json") as f:
    raw = json.load(f)

print("Sample from the dataset:")
print(json.dumps(raw[0], indent=2))


Downloading...
From: https://drive.google.com/uc?id=1qAVZOlyLXk4XCjONcyIhCkU0Buc98c2-
To: /content/data/follow-upQA.json
100%|██████████| 33.2M/33.2M [00:00<00:00, 56.4MB/s]


Sample from the dataset:
{
  "id": 0,
  "question": "Which magazine was started first Arthur's Magazine or First for Women?",
  "evidence": "Arthur's Magazine (1844\u20131846) was an American literary periodical published in Philadelphia in the 19th century.",
  "output": "First for Women is a woman's magazine published by Bauer Media Group in the USA.",
  "follow_up_question": "In which year was First for Women magazine first published?"
}


In [ ]:
# 6. Prepare and split dataset for training and validation
from datasets import Dataset, DatasetDict

def build_prompt(rec):
    # The prompt template is exactly like the Mistral
    return f"Question: {rec['question']}\nEvidence: {rec['evidence']}\nFollow-up Question:"

records = []
for r in raw:
    records.append({
        "prompt": build_prompt(r),
        "answer": r["follow_up_question"].strip()
    })

# Shuffle and split the dataset (90% train, 10% validation)
random.shuffle(records)
split = int(0.9 * len(records))
train_ds = Dataset.from_list(records[:split])
val_ds   = Dataset.from_list(records[split:])

dataset = DatasetDict({"train": train_ds, "validation": val_ds})
dataset


DatasetDict({
    train: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 45000
    })
    validation: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 5000
    })
})

In [ ]:
# 7. Load model and prepare LoRA configuration for PEFT
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# Enable gradient checkpointing to save memory
base_model.gradient_checkpointing_enable()

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

trainable params: 87,293,952 || all params: 8,278,029,312 || trainable%: 1.0545


In [ ]:
# 8. Tokenize dataset and prepare labels for language modeling
MAX_LEN = 512

def tokenize(example):
    # Tokenize the prompt + answer together
    full = tokenizer(
        example["prompt"] + " " + example["answer"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )
    # Tokenize the prompt only
    prompt_only = tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )
    prompt_len = sum(prompt_only["attention_mask"])

    # Mask labels for prompt tokens; only answer tokens are supervised
    labels = [-100] * MAX_LEN
    for i in range(prompt_len, MAX_LEN):
        if full["attention_mask"][i] == 1:          # Actual token (not padding)
            labels[i] = full["input_ids"][i]

    full["labels"] = labels
    return full

tokenized_ds = dataset.map(tokenize, remove_columns=dataset["train"].column_names,
                           batched=False, desc="Tokenizing")


Tokenizing:   0%|          | 0/45000 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# 9. Set up training arguments for model fine-tuning
from transformers import TrainingArguments, Trainer
import os

OUTPUT_DIR = "/content/Qwen3_followup_lora"
os.makedirs(OUTPUT_DIR, exist_ok=True)

EPOCHS     = 3
BATCH_SIZE = 10
GRAD_ACC   = 4
LR         = 1e-5
EVAL_STEPS = 500
SAVE_STEPS = 500

training_args = TrainingArguments(
    output_dir                  = OUTPUT_DIR,
    num_train_epochs            = EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size  = BATCH_SIZE,
    gradient_accumulation_steps = GRAD_ACC,
    learning_rate               = LR,
    warmup_ratio                = 0.03,
    logging_steps               = 100,
    eval_steps                  = EVAL_STEPS,
    save_steps                  = SAVE_STEPS,
    fp16                        = True,
    gradient_checkpointing      = True,
    push_to_hub                 = True,
    hub_model_id                = "Qwen3_finetune_follow_up_question",
    hub_private_repo            = False,
    report_to                   = "none",
)

In [ ]:
# 10. Define data collator and start model training
import torch

def collate_fn(batch):
    # Convert lists of features into PyTorch tensors
    return {
        "input_ids":      torch.tensor([ex["input_ids"]      for ex in batch], dtype=torch.long),
        "attention_mask": torch.tensor([ex["attention_mask"] for ex in batch], dtype=torch.long),
        "labels":         torch.tensor([ex["labels"]         for ex in batch], dtype=torch.long),
    }

trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = tokenized_ds["train"],
    eval_dataset  = tokenized_ds["validation"],
    data_collator = collate_fn,
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
100,1.026500
200,0.593600
300,0.570200
400,0.548100
500,0.539500
600,0.534900
700,0.532500
800,0.534300
900,0.528600
1000,0.517800


TrainOutput(global_step=3375, training_loss=0.5166755676269531, metrics={'train_runtime': 31171.1161, 'train_samples_per_second': 4.331, 'train_steps_per_second': 0.108, 'total_flos': 3.17497167839232e+18, 'train_loss': 0.5166755676269531, 'epoch': 3.0})